# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san patricio,28.0170,-97.5169,16.97,47,0,5.15,US,1741573650
1,1,fortuna,40.5982,-124.1573,12.51,63,0,6.17,US,1741573651
2,2,tiksi,71.6872,128.8694,-25.12,96,98,2.73,RU,1741573652
3,3,stanley,54.8680,-1.6985,-1.99,100,83,2.06,GB,1741573546
4,4,port-aux-francais,-49.3500,70.2167,7.43,91,100,12.58,TF,1741573654
...,...,...,...,...,...,...,...,...,...,...
565,565,puerto deseado,-47.7503,-65.8938,16.35,53,100,8.68,AR,1741574286
566,566,usogorsk,63.4106,48.6872,-10.14,98,93,1.14,RU,1741574287
567,567,vila do maio,15.1333,-23.2167,20.42,78,24,7.94,CV,1741574288
568,568,shimoda,34.6667,138.9500,10.93,54,25,3.53,JP,1741574289


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
# Configure the map plot
city_map = city_data_df.hvplot.points(
    x="Lng",  #  x-axis
    y="Lat",  # y-axis
    size="Humidity",  # Size of the point represents humidity
    color="Humidity",  # Points are colored by humidity for visual clarity
    alpha=0.6,  # Adjust transparency for overlapping points
    geo=True,  # Enable geographic plotting
    tiles=True,  
    frame_width=900,  # Set map width
    frame_height=600,  # Set map height
    title="City Humidity Levels")  # Add a title to the map


# Display the map
city_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
new_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= 20) &  # >=20°C
    (city_data_df["Max Temp"] <= 25) &  # <= 25°C
    (city_data_df["Humidity"] < 50) & 
    (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
new_cities_df = new_cities_df.dropna()

# Display sample data
new_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,60,tura,25.5198,90.2201,24.82,26,0,0.57,IN,1741573715
172,172,taoudenni,22.6783,-3.9836,22.51,28,0,4.31,ML,1741573847
227,227,san juan,-31.5375,-68.5364,20.80,42,0,3.74,AR,1741573202
270,270,kerikeri,-35.2268,173.9474,22.43,42,0,5.22,NZ,1741573958
313,313,three points,32.0767,-111.3137,20.77,13,0,3.09,US,1741574005
317,317,las veredas,23.1388,-109.7075,20.25,44,0,2.57,MX,1741574011
323,323,illizi,26.4833,8.4667,20.25,19,0,2.52,DZ,1741574017
336,336,cabo san lucas,22.8909,-109.9124,21.17,45,0,2.57,MX,1741574033
551,551,sao vicente,-23.9631,-46.3919,24.43,26,0,1.60,BR,1741574271


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
60,tura,IN,25.5198,90.2201,26,
172,taoudenni,ML,22.6783,-3.9836,28,
227,san juan,AR,-31.5375,-68.5364,42,
270,kerikeri,NZ,-35.2268,173.9474,42,
313,three points,US,32.0767,-111.3137,13,
317,las veredas,MX,23.1388,-109.7075,44,
323,illizi,DZ,26.4833,8.4667,19,
336,cabo san lucas,MX,22.8909,-109.9124,45,
551,sao vicente,BR,-23.9631,-46.3919,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters (10 km)
params = {
    "categories": "accommodation.hotel",  # Category for hotels
    "limit": 1,  # Limit to the first result
    "apiKey": geoapify_key  # Your Geoapify API key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]


    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"  # Circle filter: center at city coordinates
    params["bias"] = f"proximity:{longitude},{latitude}"  # Bias search to prioritize close results


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tura - nearest hotel: Hotel Polo Orchid
taoudenni - nearest hotel: No hotel found
san juan - nearest hotel: Hotel San Francisco
kerikeri - nearest hotel: Avalon Resort
three points - nearest hotel: No hotel found
las veredas - nearest hotel: Hotel Aeropuerto
illizi - nearest hotel: بوناقة
cabo san lucas - nearest hotel: Comfort Rooms
sao vicente - nearest hotel: Pousada Vitória


,City,Country,Lat,Lng,Humidity,Hotel Name
60,tura,IN,25.5198,90.2201,26,Hotel Polo Orchid
172,taoudenni,ML,22.6783,-3.9836,28,No hotel found
227,san juan,AR,-31.5375,-68.5364,42,Hotel San Francisco
270,kerikeri,NZ,-35.2268,173.9474,42,Avalon Resort
313,three points,US,32.0767,-111.3137,13,No hotel found
317,las veredas,MX,23.1388,-109.7075,44,Hotel Aeropuerto
323,illizi,DZ,26.4833,8.4667,19,بوناقة
336,cabo san lucas,MX,22.8909,-109.9124,45,Comfort Rooms
551,sao vicente,BR,-23.9631,-46.3919,26,Pousada Vitória


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot - Taking from https://hvplot.holoviz.org/user_guide/Geographic_Data.html
city_map = hotel_df.hvplot.points(
    x="Lng",  # Longitude  x-axis
    y="Lat",  # Latitude y-axis
    size="Humidity",  # Size of the point represents humidity
    color="Humidity",  # Points are colored by humidity
    geo=True,  
    tiles=True ,
    frame_width=900,  # Set the map width
    frame_height=600,  # Set the map height
    title="City Humidity Levels with Hotel Info",  # Add a descriptive title
    hover_cols=["City", "Country", "Hotel Name"]  # Include additional columns in hover message
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)